In [1]:
from illufly.types import Message, Messages, Template

## role 生成规则

### 规则1： 不指定时，首条是 system，system 后接 user，user 与 assistant 交替

In [2]:
Messages(["hi"]).to_list()

[{'role': 'system', 'content': 'hi'}]

In [3]:
Messages(["hi", "我很开心", "我也很开心"]).to_list()

[{'role': 'system', 'content': 'hi'},
 {'role': 'user', 'content': '我很开心'},
 {'role': 'assistant', 'content': '我也很开心'}]

In [4]:
Messages([
    Template(text="你是一个小说家。"),
    "帮我创作吧",
    "从前有一个人很坏，他坏死了。\n额，我是他说真的死了。"
]).to_list()

[{'role': 'system', 'content': '你是一个小说家。'},
 {'role': 'user', 'content': '帮我创作吧'},
 {'role': 'assistant', 'content': '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'}]

In [5]:
Messages([
    ("system", Template(text="你是一个小说家，帮我创作吧。")),
    "我这个小说很有趣哦，我结束了",
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '我这个小说很有趣哦，我结束了'}]

In [6]:
Messages([
    ("user", Template(text="你是一个小说家，帮我创作一个说。")),
    {"role": "assistant", "content": "我开始了"},
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '我开始了'}]

### 规则2：列表中允许出现字符串、Turple、Template、Message、Dict等类型

In [31]:
Messages([
    Template(text="你是一个小说家，帮我创作吧。"),
    ("user", "请你开始")
]).raw_messages

[<Template consumer_dict=set() text='你是一个小说家，帮我创作吧。'>, ('user', '请你开始')]

In [32]:
Messages([
    Template(text="你是一个小说家，帮我创作吧。"),
    ("user", "请你开始")
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '请你开始'}]

### 规则3：允许用 ai 替代 assistant 作为 role 来声明，但会被替换为 assistant

In [33]:
Messages([
    ("user", Template(text="你是一个小说家，帮我创作一个说。")),
    ("ai", "请你开始")
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '请你开始'}]

## 多模态消息

### 多模态消息的定义兼容原有格式

In [34]:
Messages([
    ("user", Template(text="你是一个小说家，帮我创作一个说。")),
    ("ai", "我开始了"),
    {"role": "user", "content": "好"},
    {"role": "ai", "content": "我真的开始了"},
]).to_list(input_vars={}, style="qwen_vl")

[{'role': 'user', 'content': [{'text': '你是一个小说家，帮我创作一个说。'}]},
 {'role': 'assistant', 'content': [{'text': '我开始了'}]},
 {'role': 'user', 'content': [{'text': '好'}]},
 {'role': 'assistant', 'content': [{'text': '我真的开始了'}]}]

In [35]:
Template(text="你是一个小说家，帮我创作一个说。").format()

'你是一个小说家，帮我创作一个说。'

### OpenAI风格的多模态消息格式

In [36]:
openai_msg = Messages(
    [
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png"
              }
            },
            {
              "type": "text",
              "text": "这些是什么"
            }
          ]
        }
    ],
    style="openai_vl"
)

In [37]:
openai_msg.to_list()

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'}},
   {'type': 'text', 'text': '这些是什么'}]}]

In [38]:
openai_msg.to_list(style="qwen_vl")

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]}]

In [39]:
openai_msg.to_list(style="text")

[{'role': 'user', 'content': '这些是什么'}]

### 通义千问的多模态消息格式

In [40]:
qwen_msg = Messages([
    ("user", [
        {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
        {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
        {'text': '这些是什么'}
    ])
], style="qwen_vl")

In [41]:
qwen_msg.to_list()

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]}]

In [42]:
qwen_msg.to_list(style="openai_vl")

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'}},
   {'type': 'text', 'text': '这些是什么'}]}]

In [43]:
qwen_msg.to_list(style="text")

[{'role': 'user', 'content': '这些是什么'}]

## 工具回调消息

当消息列表中包含工具回调消息时，要求兼容：

### 在消息列表中兼容工具消息

In [10]:
from illufly.types import Messages

messages = Messages([{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'role': 'tool',
  'name': 'get_current_time',
  'content': '当前时间：2024-09-28 14:33:56。'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}])

messages.to_list()

[{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'role': 'tool',
  'content': '当前时间：2024-09-28 14:33:56。',
  'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'name': 'get_current_time'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}]

### 从文本消息转换为多模态消息

In [11]:
messages.to_list(style="openai_vl")

[{'role': 'user', 'content': [{'type': 'text', 'text': '几点了'}]},
 {'role': 'assistant',
  'content': [{'type': 'text', 'text': ''}],
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'role': 'tool',
  'content': [{'type': 'text', 'text': '当前时间：2024-09-28 14:33:56。'}],
  'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'name': 'get_current_time'},
 {'role': 'assistant',
  'content': [{'type': 'text', 'text': '现在的时间是2024年9月28日 14点33分56秒。'}]},
 {'role': 'user', 'content': [{'type': 'text', 'text': '现在几点了？'}]}]

In [12]:
messages.to_list(style="qwen_vl")

[{'role': 'user', 'content': [{'text': '几点了'}]},
 {'role': 'assistant',
  'content': [{'text': ''}],
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'role': 'tool',
  'content': [{'text': '当前时间：2024-09-28 14:33:56。'}],
  'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'name': 'get_current_time'},
 {'role': 'assistant', 'content': [{'text': '现在的时间是2024年9月28日 14点33分56秒。'}]},
 {'role': 'user', 'content': [{'text': '现在几点了？'}]}]

In [13]:
messages.raw_messages

[{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'role': 'tool',
  'name': 'get_current_time',
  'content': '当前时间：2024-09-28 14:33:56。'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}]

## 在消息列表中包含提示语模板

In [1]:
from illufly.types import Message, Messages, Template

In [2]:
msgs = Messages([
    Template(text="你是一个小说家，帮我创作吧。"),
    ("user", "不错"),
])
msgs.to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '不错'}]

In [4]:
msgs.has_role("tool")

False

### 模板输入变量映射

In [46]:
Messages([
    Template(text="你是一个小说家，帮我创作{{task}}吧。"),
    ("user", "请开始")
]).to_list({"task": "小说"})

[{'role': 'system', 'content': '你是一个小说家，帮我创作小说吧。'},
 {'role': 'user', 'content': '请开始'}]

### 使用绑定映射模板变量

In [54]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str, **kwargs):
        self._last_input = prompt

# 模板已经声明
t = Template(text="你是一个小说家，帮我创作{{last_input}}吧。")

# 完成绑定
a = A()
a.bind_consumers(t)

# 动态修改了对象的 last_input 属性
a("帮我写一个小说")

# 模板通过绑定获得 last_input 值
Messages([t, "请开始"]).to_list({"task": "小说"})

[{'role': 'system', 'content': '你是一个小说家，帮我创作帮我写一个小说吧。'},
 {'role': 'user', 'content': '请开始'}]

In [56]:
a.provider_dict

{'last_input': '帮我写一个小说'}

In [57]:
t.consumer_dict

{'last_input': '帮我写一个小说'}